In [1]:
import numpy as np
import pylab as plt
import networkx as nx
import scipy.integrate as integ
from scipy.linalg import null_space
import math

from utils import *

%matplotlib inline

In [2]:
def simplicial_kuramoto_full_theta(t, theta, B0, B1, a, omega_0):#, weights_n, weights_e, weights_f
    return omega_0-a*(B0.dot(np.sin(B0.T.dot(theta))))-a*B1.T.dot(np.sin(B1.dot(theta)))
#     return omega_0-a*np.diag(we).dot(B0.dot(np.sin(B0.T.dot(theta))))-a*B1.T.dot(np.diag(wf).dot(np.sin(B1.dot(theta))))

def integrate_simplicial_kuramoto_full_theta(B0, B1, theta_0, t_max, n_t, a, omega_0):#, weights_n, weights_e, weights_f):
    return integ.solve_ivp(lambda t, theta: simplicial_kuramoto_full_theta(t, theta, B0, B1, a, omega_0), [0, t_max], theta_0, t_eval = np.linspace(0, t_max, n_t),method='Radau',rtol=1.49012e-8,atol=1.49012e-8)
#     return integ.solve_ivp(lambda t, theta: simplicial_kuramoto_full_theta(t, theta, B0, B1, a, omega_0, degree, weights_n, weights_e, weights_f), [0, t_max], theta_0, t_eval = np.linspace(0, t_max, n_t),method='Radau',rtol=1.49012e-8,atol=1.49012e-8)

In [3]:
def ntri(A):
    # signed incidence matrices for triangles
    Nn=A.shape[0]
    Ne=int(np.sum(A)/2)
    #print Nn, Ne

    e=np.zeros((Ne,2))
    count=0;
    for i in range(Nn):
        for j in range(i+1,Nn):
            if(A[i,j]>0):
                e[count,0]=i
                e[count,1]=j
                count+=1
    print "edges"
    print e
    I=np.zeros((Ne,Nn))
    for i in range(Ne):
        I[i,int(e[i,0])]=1
        I[i,int(e[i,1])]=-1
    #print I

    Nf=0
    for i in range(Nn):
        for j in range(i+1,Nn):
            for k in range(j+1,Nn):
                subA=A[np.ix_([i,j,k],[i,j,k])]
                if(np.sum(subA)==6):
                    Nf+=1
    f=np.zeros((Nf,3))
    count=0
    for i in range(Nn):
        for j in range(i+1,Nn):
            for k in range(j+1,Nn):
                subA=A[np.ix_([i,j,k],[i,j,k])]
                if(np.sum(subA)==6):
                    f[count,0]=i
                    f[count,1]=j
                    f[count,2]=k
                    count+=1
    print "faces"
    print f
    II=np.zeros((Nf,Ne))
    for i in range(f.shape[0]):
        for j in [0,-1,-2]:
            temp=np.roll(f[i,:],j)
            temp=temp[0:2]
            for k in range(e.shape[0]):
                #print e[k,:],temp
                if(((e[k,:]==temp).all())or((e[k,:]==np.roll(temp,1)).all())):
                    Irow=k
            if(temp[0]<temp[1]):
                II[i,Irow]=1
            else:
                II[i,Irow]=-1
    #print II 
    ntrie=np.sum(II,1)
    return I,II#,ntrie, e#, len(ntrie)

In [4]:
# cycle graph
Nn=4

G=nx.cycle_graph(Nn)

A=nx.to_numpy_matrix(G)

B0,B1=ntri(A)

Nn=B0.shape[1]
Ne=B0.shape[0]
Nf=B1.shape[0]

# inital conditions
# np.random.seed(seed=4444)
theta_0=2*np.pi*np.random.rand(Ne)
# theta_0=(np.random.rand(Ne)-0.5)*10

theta_final=plotflow(theta_0,B0,B1,'One opposite orientation')

# reverse the orientation of the odd one out
B0[1,0]=-B0[1,0]
B0[1,-1]=-B0[1,-1]

plotflow(theta_0,B0,B1,'All same orientations')
plotflow(theta_final,B0,B1,'All same orientations') #to check that the stationary state with the odd one out is unstable for the all the same

edges
[[0. 1.]
 [0. 3.]
 [1. 2.]
 [2. 3.]]
faces
[]


NameError: name 'plotflow' is not defined

## Line graph

In [ ]:
def plotflow(theta_0,B0,B1,plotname):
    print(plotname)
    omega_0=np.ones(B0.shape[0])*0
#     degree=np.absolute(B0).sum(0)
    a=10
    t_max = 110 #integration time
    n_t = 200 #number of timepoints 

    result=integrate_simplicial_kuramoto_full_theta(B0, B1, theta_0, t_max, n_t, a, omega_0)
    times = result.t
    theta = result.y
    
    plt.figure()
    plt.imshow(np.mod(np.around(theta,10),np.around(2*np.pi,10)), aspect='auto',cmap='bwr')
    plt.title(plotname+' phases')
    plt.colorbar()
    
#     op=order_parameter(theta, 4, 1)
    op=order_parameter(np.mod(np.around(theta,10),np.around(2*np.pi,10)), 4, 1)
    plt.figure()
    plt.title(plotname+' order parameter')
    plt.plot(op[0,:])
    
    print('\theta_0: ', theta_0)
    print('\theta_final: ',theta[:,-1])
    
    Div=np.mod(np.around(B0.T.dot(theta),10),np.around(2*np.pi,10))
    Curl=np.mod(np.around(B1.dot(theta),10),np.around(2*np.pi,10))
    print('Div: ', Div[:,-1])
    print('Curl: ', Curl[:,-1])
    
    L1=-B0.dot(B0.T)-B1.T.dot(B1)
    print('L1\theta: ', L1.dot(theta[:,-1]))
    print('L1\theta (mod 2pi): ', np.mod(np.around(L1.dot(theta[:,-1]),10),np.around(2*np.pi,10)))
    print('dim(Ker(L1)): ', null_space(L1).shape[1])
    print('Ker(L1): ', null_space(L1))
    
#     plt.figure()
#     plt.imshow(Div, aspect='auto',cmap='bwr')
#     plt.title(plotname+' divergence')
#     plt.colorbar()
#     plt.figure()
#     plt.imshow(Curl, aspect='auto',cmap='bwr')
#     plt.title(plotname+' curl')
#     plt.colorbar()
    return theta[:,-1]

In [ ]:
# barbell with two 4-cycles
Nn=8
Ne=9
Nf=0
B0=np.zeros((Ne,Nn))
B0[0,:]=[-1, 1, 0, 0, 0, 0, 0, 0]
B0[1,:]=[0, -1, 1, 0, 0, 0, 0, 0]
B0[2,:]=[0, 0, -1, 1, 0, 0, 0, 0]
B0[3,:]=[1, 0, 0, -1, 0, 0, 0, 0]
B0[4,:]=[0, 0, 0, 0, -1, 1, 0, 0]
B0[5,:]=[0, 0, 0, 0, 0, -1, 1, 0]
B0[6,:]=[0, 0, 0, 0, 0, 0, -1, 1]
B0[7,:]=[0, 0, 0, 0, 1, 0, 0, -1]
B0[8,:]=[0, 1, 0, 0, 0, 0, 0, -1] #barbell
B1=np.zeros((Nf,Ne))

# 4-cycle
# Nn=4
# G=nx.cycle_graph(Nn)
# A=nx.to_numpy_matrix(G)
# B0,B1=ntri(A)

G=nx.from_numpy_matrix(B0.T.dot(B0)-np.diag(np.diag(B0.T.dot(B0))))
B0abs=np.abs(B0)
# Gedge=nx.from_numpy_matrix(B0.dot(B0.T)-np.diag(np.diag(B0.dot(B0.T))))
Gedge=nx.from_numpy_matrix(B0abs.dot(B0abs.T)-np.diag(np.diag(B0abs.dot(B0abs.T))))
Aedge=np.abs(B0.dot(B0.T)-np.diag(np.diag(B0.dot(B0.T))))
Gline=nx.line_graph(G)
Aline=nx.to_numpy_matrix(Gline)
# print('Adjacency edge: ',Aedge, Aedge.sum(0))
# print('Adjacency line: ',Aline, Aline.sum(0))

B0_2=np.asarray(nx.incidence_matrix(Gedge,oriented=False).todense())
B0_2_or=np.asarray(nx.incidence_matrix(Gedge,oriented=True).todense())
I0=np.asarray(nx.incidence_matrix(Gline,oriented=False).todense())
I0_or=np.asarray(nx.incidence_matrix(Gline,oriented=True).todense())
print('Incidence from adjacency edge',B0_2_or,B0_2_or.shape)
print('Incidence from adjacency line',I0_or,I0_or.sum(1))

# different graphs plots
plt.figure()
pos=nx.spring_layout(Gedge)
nx.draw_networkx(G, pos=pos)
plt.title('G')
plt.show()

plt.figure()
pos=nx.spring_layout(Gedge)
nx.draw_networkx(Gedge,pos=pos)
plt.title('Gedge')
plt.show()

plt.figure()
pos=nx.spring_layout(Gline)
nx.draw_networkx(Gline,pos=pos)
plt.title('Gline')
plt.show()

print(B0.shape,B0_2.shape,I0.shape)

In [ ]:
# inital conditions
# np.random.seed(seed=4444)

theta_0=2*np.pi*np.random.rand(Ne)

# theta_0=(np.random.rand(Ne)-0.5)*10
Ne=B0.shape[0]
B1=np.zeros((Nf,Ne))

theta_0=2*np.pi*np.random.rand(Ne)
theta_final_edge=plotflow(theta_0,B0,B1,'Incidence from the graph, oriented')
theta_final_line=plotflow(theta_0,I0_or,B1,'Incidence from the line graph, oriented')
theta_final_edge=plotflow(theta_0,B0_2_or,B1,'Incidence from the edge graph, oriented')

# Ne=I0.shape[1]
# B1=np.zeros((Nf,Ne))
# theta_0=2*np.pi*np.random.rand(Ne)
# theta_final_line=plotflow(theta_0,I0_or.T,B1,'Incidence from the line graph, oriented')
# theta_final_edge=plotflow(theta_0,B0_2_or.T,B1,'Incidence from the edge graph, oriented')
# theta_final_line=plotflow(theta_0,I0,B1,'Incidence from the line graph')
# theta_final_edge=plotflow(theta_0,B0_2,B1,'Incidence from the edge graph')